# LAB4

#### Функция чтения аннотации

In [8]:
import csv
import typing
import pandas as pd
import cv2 

In [ ]:
def read_file(path: str) -> list[list[str]]:
    '''Читает файл path и возвращает матрицу имён файлов'''
    files: list[list[str]] = []
    csvfile: typing.TextIO = open(path, "r")
    reader: csv.DictReader = csv.DictReader(csvfile, delimiter=",")
    for row in reader:
        files.append([row["full_path"], row["class"]])
    return files

list_file_info: list[list[str]] = read_file("annotation.csv")

#### (1) Создание датафрейма с двумя колонками

In [ ]:
def create_df_with_two_columns(list_files_info):
    df = pd.DataFrame(columns=["Name", "AbsPath"])
    for file_info in list_files_info:
        df.loc[len(df)] = [file_info[1], file_info[0]]
    return df

create_df_with_two_columns(list_file_info)

#### (2,3,4) Именование колонок, метка, форма

In [9]:
def create_df(list_files_info):
    df = pd.DataFrame(columns=["Name", "AbsPath", "ClassId", "Weight", "Height", "Depth"])
    for file_info in list_file_info:
        class_id = (0 if "cat" == file_info[1] else 1)
        im = cv2.imread(file_info[0])
        h,w,c = im.shape
        df.loc[len(df)] = [file_info[1], file_info[0], class_id, w,h,c]
    return df

df = create_df(list_file_info)
df

,Name,AbsPath,ClassId,Weight,Height,Depth
0,cat,C:\Users\Professional\Desktop\lab python\datas...,0,427,320,3
1,cat,C:\Users\Professional\Desktop\lab python\datas...,0,444,320,3
2,cat,C:\Users\Professional\Desktop\lab python\datas...,0,480,320,3
3,cat,C:\Users\Professional\Desktop\lab python\datas...,0,480,300,3
4,cat,C:\Users\Professional\Desktop\lab python\datas...,0,480,300,3
...,...,...,...,...,...,...
1995,dog,C:\Users\Professional\Desktop\lab python\datas...,1,466,320,3
1996,dog,C:\Users\Professional\Desktop\lab python\datas...,1,455,320,3
1997,dog,C:\Users\Professional\Desktop\lab python\datas...,1,480,320,3
1998,dog,C:\Users\Professional\Desktop\lab python\datas...,1,480,320,3
